In [1]:
from IPython.display import clear_output
import numpy as np

In [40]:
DATA_COUNT = 1000
DIM = 3
DATA_LIMIT = 10

Rotation stuffs

In [3]:
def rotate_vec(dim=4, theta=None, i=None, j=None):

    if theta is None:
        theta = 2 * np.pi * np.random.random()

    if i is None:
        i = np.random.choice(dim)

    if j is None:
        j = np.random.choice(dim)
        while i == j:
            j = np.random.choice(dim)

    rotational_matrix = np.identity(dim)
    rotational_matrix[i][i] = rotational_matrix[j][j] = np.cos(theta)
    rotational_matrix[i][j] = np.sin(theta)
    rotational_matrix[j][i] = -np.sin(theta)

    return rotational_matrix


def random_rot(dim, count=10):
    id = np.identity(dim)
    for i in range(count):
        id = id.dot(rotate_vec(dim=dim))
    return id


Creating data

In [82]:


def tensor_diff(a, b):
    c = a - b
    c = np.squeeze(c)
    c_t = np.var(c)
    c = np.square(c)
    c = np.sqrt(np.average(c))
    return np.average(c), c_t, np.exp(- np.average(c)) * 100

def dropout_p_matrix(percent,shp, step = True):
    ch = np.random.random(shp[0])
    d_l = np.vectorize(lambda x : 1 if x < percent else 0)
    x = d_l(ch)
    return np.tile(x, (shp[1], 1)).T

def hyp(vex):
    return vex

In [5]:
np.set_printoptions(precision=4)
np.set_printoptions(suppress=True)

In [6]:

def regressutil(weights, training_data, training_label, epochs, testing_data, testing_label,alpha = 0.001, update_interval = 10, retain = 1):

    for epoch in range(epochs):        
        hidden_layer = training_data.dot(weights[0] * dropout_p_matrix(retain, weights[0].shape))
        predicted_label = hidden_layer.dot(weights[1]* dropout_p_matrix(retain, weights[1].shape))
        error = training_label - predicted_label
        dW0 = - training_data.T.dot(2 * error.dot(weights[1].T)) / training_data.size
        dW1 = - hidden_layer.T.dot(2 * error) / training_data.size
        weights[0] -= dW0 * alpha
        weights[1] -= dW1 * alpha

        if epoch % (epochs // update_interval) == 0:
            clear_output()
            print("Error :", tensor_diff(testing_data.dot(weights[0]).dot(weights[1]), testing_label))
            print("Weight")
            print(weights)
    return weights

        

In [30]:
def regress(W, vecs, vecsy, epochs, initial_dropout = 0.7):
    vecs = np.array(np.split(vecs, 10))
    vecsy = np.array(np.split(vecsy, 10))
    for i in range(10):
        W = regressutil(W, np.vstack(np.delete(vecs, i, axis=0)), np.vstack(np.delete(vecsy, i, axis=0)), epochs //10, vecs[i], vecsy[i], update_interval = 1, retain = initial_dropout)
    
    return W

In [97]:
W = np.array([np.random.random((DIM + 1, DIM + 1)), np.random.random((DIM + 1, DIM + 1))])
vecs = (np.random.random((DATA_COUNT, DIM)) * 2 * DATA_LIMIT) - DATA_LIMIT
vecs = np.vstack((vecs.T, np.zeros(DATA_COUNT))).T
print(vecs)
r = random_rot(DIM + 1, count=0)
vecs = vecs.dot(r)
vecsy = vecs


[[-3.2198 -9.5895  6.9631  0.    ]
 [ 8.3814  8.6646 -9.1993  0.    ]
 [-6.356  -1.8111 -6.5993  0.    ]
 ...
 [-8.339   1.6638  9.6059  0.    ]
 [-3.0417 -4.5368 -5.4165  0.    ]
 [ 9.7146 -8.5866  2.6064  0.    ]]


In [ ]:
W = regressutil(W, vecs, vecsy, 80000, vecs, vecsy, update_interval = 10, retain = 1, alpha= 0.001)
W

In [100]:
v1 = (np.random.random((DATA_COUNT, DIM)) * 2 * DATA_LIMIT) - DATA_LIMIT
v1 = np.vstack((v1.T, np.zeros(DATA_COUNT))).T
v2 = v1.dot(random_rot(DIM + 1))
print(tensor_diff(vecs.dot(W[0]).dot(W[1]), vecs))
tensor_diff(v2.dot(W[0]).dot(W[1]), v2)

(7.285579595611618, 53.07672610291935, 0.06853508948451276)


(6.1406616769200815, 37.577431603571284, 0.2153498226701214)

In [99]:
W = regress(W, vecs, vecsy, 800000, initial_dropout=0.8)
W

Error : (3.388175163332776, 11.46545907447357, 3.3770245888771333)
Weight
[[[-2.9276 -3.4503  0.5088  7.4771]
  [-1.1245  4.9676 -6.8     1.9254]
  [ 6.865  -4.2611 -3.7866  0.8312]
  [ 0.8531  0.0445  0.6424  0.7249]]

 [[-0.0765 -0.0509  0.0873  0.0052]
  [-0.0903  0.0575 -0.0464  0.0047]
  [ 0.0039 -0.0844 -0.0515  0.0052]
  [ 0.2051  0.0746  0.0041  0.0075]]]


array([[[-2.9192, -3.4938,  0.5585,  7.465 ],
        [-1.0444,  5.1003, -6.9969,  1.866 ],
        [ 6.9518, -4.2963, -3.913 ,  0.8973],
        [ 0.8531,  0.0445,  0.6424,  0.7249]],

       [[-0.1266,  0.0283,  0.1913,  0.0046],
        [-0.0377,  0.0976, -0.0797,  0.0004],
        [-0.0021, -0.2601, -0.1574,  0.0128],
        [ 0.2055,  0.1042,  0.026 ,  0.0055]]])